In [52]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT
import pickle
from time import time

from op_fourier_trafo import *
from boltzmann import *
from liouv_step import *
from tools.classical import *
from tools.quantum import *

In [53]:
sigmax = np.array([[0, 1], [1, 0]])
sigmay = np.array([[0, -1j], [1j, 0]])
sigma_minus = (sigmax - 1j * sigmay) / 2
print(f'Sigma minus: \n{sigma_minus}')
term_coeffs = np.array([0.5, -1j*0.5])
sqrt_term_coeffs = np.sqrt(term_coeffs / np.sum(np.abs(term_coeffs)))
print(sqrt_term_coeffs)

# Unitary that brings ancilla into right state for LCU 
B = np.array([[sqrt_term_coeffs[0], sqrt_term_coeffs[1].conj()], [sqrt_term_coeffs[1], -sqrt_term_coeffs[0]]])
B_dag = B.conj().T
print('PREPs')
print(B)
print(B_dag)
B_op = Operator(B)

ini_state = np.array([1, 0])
one_state = np.array([0, 1])
end_state = B @ ini_state
end_one_state = B @ one_state
print(end_state)
print(f'Prep on 1: {end_one_state}')
zero_state = np.array([1, 0])
actual_proj0 = np.kron(np.eye(2), zero_state)
print(actual_proj0)
print(np.kron(zero_state, np.eye(2)).reshape(4, 2))
print(np.sqrt(0.5 * 1j))


Sigma minus: 
[[0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j]]
[0.707+0.j  0.5  -0.5j]
PREPs
[[ 0.707+0.j   0.5  +0.5j]
 [ 0.5  -0.5j -0.707-0.j ]]
[[ 0.707-0.j   0.5  +0.5j]
 [ 0.5  -0.5j -0.707+0.j ]]
[0.707+0.j  0.5  -0.5j]
Prep on 1: [ 0.5  +0.5j -0.707+0.j ]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0.]
 [0. 0.]
 [0. 1.]
 [0. 0.]]
(0.5+0.5j)


In [54]:
num_qubits = 1
num_block_encoding_qubits = 1
qr_sys = QuantumRegister(num_qubits, 'sys')
qr_block = QuantumRegister(num_block_encoding_qubits, 'block')
circ = QuantumCircuit(qr_sys, qr_block)  #! Order matters
#TODO: figure out how to apply this block such that it gets the system in the right state! Test it!
#* Prep
circ.append(B_op, [qr_block[0]])

#* Select
# Pick 0th term
circ.x(qr_block)
circ.cx(qr_block, qr_sys)
circ.x(qr_block)

# Pick 1st term
circ.cy(qr_block, qr_sys)

#* Prep dagger
circ.append(B_op.transpose(), [qr_block[0]])
print(circ)

circ_op = Operator(circ)
statevector = Statevector(circ)
np.set_printoptions(precision=3, suppress=True)
print(circ_op.data)



                       ┌───┐     ┌───┐           
  sys: ────────────────┤ X ├─────┤ Y ├───────────
       ┌─────────┐┌───┐└─┬─┘┌───┐└─┬─┘┌─────────┐
block: ┤ Unitary ├┤ X ├──■──┤ X ├──■──┤ Unitary ├
       └─────────┘└───┘     └───┘     └─────────┘
[[0.   +0.j    0.   +0.j    0.   +0.j    0.707+0.707j]
 [1.   +0.j    0.   +0.j    0.   +0.j    0.   +0.j   ]
 [0.   +0.j    0.707+0.707j 0.   +0.j    0.   -0.j   ]
 [0.   +0.j    0.   +0.j    0.   +1.j    0.   +0.j   ]]


In [55]:
right_proj = np.array([[1, 0], [0, 1], [0, 0], [0, 0]])
left_proj = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
print(right_proj)
print(left_proj)
print(circ_op.data.shape)
block_extracted = left_proj @ circ_op.data @ right_proj
block_extracted

[[1 0]
 [0 1]
 [0 0]
 [0 0]]
[[1 0 0 0]
 [0 1 0 0]]
(4, 4)


array([[0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j]])